In [14]:
!pip install sqlalchemy
import pandas as pd
from sqlalchemy import create_engine

In [15]:
# pull first csv into dataframe
min_wage_file = "Raw_Data/Minimum_Wage_Data.csv"
min_wage_df = pd.read_csv(min_wage_file, encoding = "utf-8")
min_wage_df.head()

,Year,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.1,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


In [16]:
# remove years prior to 2010
min_year_df = min_wage_df.loc[(min_wage_df['Year'] == 2010) |
                          (min_wage_df['Year'] == 2011) |
                          (min_wage_df['Year'] == 2012) |
                          (min_wage_df['Year'] == 2013) |
                          (min_wage_df['Year'] == 2014) |
                          (min_wage_df['Year'] == 2015) |
                          (min_wage_df['Year'] == 2016)]

# alternative:
# min_wage_df2 = min_wage_df[(min_wage_df[‘Year’] > 2009)]
# min_wage_df2.head()

min_year_df

,Year,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
2310,2010,Alabama,...,NaN,0.00,0.00,218.055500,0.00,0.00
2311,2010,Alaska,7.75,NaN,7.75,7.75,218.055500,8.90,8.90
2312,2010,Arizona,7.25,NaN,7.25,7.25,218.055500,8.33,8.33
2313,2010,Arkansas,6.25[c],[c],6.25,6.25,218.055500,7.18,7.18
2314,2010,California,8,NaN,8.00,8.00,218.055500,9.19,9.19
2315,2010,Colorado,7.24,NaN,7.24,7.24,218.055500,8.31,8.31
2316,2010,Connecticut,8.25,NaN,8.25,8.25,218.055500,9.47,9.47
2317,2010,Delaware,7.25,NaN,7.25,7.25,218.055500,8.33,8.33
2318,2010,District of Columbia,8.25,NaN,8.25,8.25,218.055500,9.47,9.47
2319,2010,Federal (FLSA),7.25,NaN,7.25,7.25,218.055500,8.33,8.33


In [17]:
# remove extraneous columns
min_wage_df2 = min_year_df.drop(['Footnote', 'Table_Data', 'High.2018', 'Low.2018', 'CPI.Average'], axis =1)
min_wage_df2.head()

,Year,State,High.Value,Low.Value
2310,2010,Alabama,0.00,0.00
2311,2010,Alaska,7.75,7.75
2312,2010,Arizona,7.25,7.25
2313,2010,Arkansas,6.25,6.25
2314,2010,California,8.00,8.00


In [18]:
# make column names lower-case
min_wage_df2.columns = ["year","state","high_value","low_value"]
min_wage_df2.head()

,year,state,high_value,low_value
2310,2010,Alabama,0.00,0.00
2311,2010,Alaska,7.75,7.75
2312,2010,Arizona,7.25,7.25
2313,2010,Arkansas,6.25,6.25
2314,2010,California,8.00,8.00


In [50]:
# Make the index the id column
min_wage_df2 = min_wage_df2.reset_index(drop=False)
min_wage_df2.rename(columns = {"index":"id"}, inplace = True)
min_wage_df2

,id,year,state,high_value,low_value
0,2310,2010,Alabama,0.00,0.00
1,2311,2010,Alaska,7.75,7.75
2,2312,2010,Arizona,7.25,7.25
3,2313,2010,Arkansas,6.25,6.25
4,2314,2010,California,8.00,8.00
5,2315,2010,Colorado,7.24,7.24
6,2316,2010,Connecticut,8.25,8.25
7,2317,2010,Delaware,7.25,7.25
8,2318,2010,District of Columbia,8.25,8.25
9,2319,2010,Federal (FLSA),7.25,7.25


In [19]:
# pull second csv into dataframe
pub_assistance_file = "Raw_Data/sub-est2016_all.csv"
pub_assistance_df = pd.read_csv(pub_assistance_file, encoding='utf-8')
pub_assistance_df.head()

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2688,2683,2685,2647,2631,2619,2616,2603
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4522,4517,4495,4472,4447,4428,4395,4360
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,756,754,753,748,748,747,740,738
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,345,345,342,337,337,334


In [20]:
# rename columns to be more readable
pub_assistance_df.columns = ["geographic_summary_level","state_FIPS_code", "county_FIPS_code", "place_FIPS_code", "minor_civil_div_FIPS_code", "consolidated_city_FIPS_code", "primitive_geography_flag", "functional_status_code","city", "state", "census_pop_2010", 
                             "est_base_2010", "est_pop_2010", "est_pop_2011", "est_pop_2012", "est_pop_2013", "est_pop_2014", "est_pop_2015", "est_pop_2016"]

pub_assistance_df.head()

,geographic_summary_level,state_FIPS_code,county_FIPS_code,place_FIPS_code,minor_civil_div_FIPS_code,consolidated_city_FIPS_code,primitive_geography_flag,functional_status_code,city,state,census_pop_2010,est_base_2010,est_pop_2010,est_pop_2011,est_pop_2012,est_pop_2013,est_pop_2014,est_pop_2015,est_pop_2016
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2688,2683,2685,2647,2631,2619,2616,2603
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4522,4517,4495,4472,4447,4428,4395,4360
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,756,754,753,748,748,747,740,738
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,345,345,342,337,337,334


In [21]:
# create new dictionary to replace SUMLEV column numbers with their description
SUMLEV_dict = {"geographic_summary_level": [40,50,61,71,157,162,170,172],
               "geographic_level": ["State", "County", "Minor Civil Division", "Minor Civil Division place part",
                                    "County place part", "Incorporated place", "Consolidated city",
                                    "Consolidated city -- place within consolidated city"]}

SUMLEV_df = pd.DataFrame(SUMLEV_dict)

In [22]:
# merge dictionary with dataframe as new column
new_pub_assist_df = pub_assistance_df.merge(SUMLEV_df, on='geographic_summary_level', how='left')
new_pub_assist_df.head()

,geographic_summary_level,state_FIPS_code,county_FIPS_code,place_FIPS_code,minor_civil_div_FIPS_code,consolidated_city_FIPS_code,primitive_geography_flag,functional_status_code,city,state,census_pop_2010,est_base_2010,est_pop_2010,est_pop_2011,est_pop_2012,est_pop_2013,est_pop_2014,est_pop_2015,est_pop_2016,geographic_level
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300,State
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2688,2683,2685,2647,2631,2619,2616,2603,Incorporated place
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4522,4517,4495,4472,4447,4428,4395,4360,Incorporated place
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,756,754,753,748,748,747,740,738,Incorporated place
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,345,345,342,337,337,334,Incorporated place


In [23]:
# pull out state entries
state_pub_assist_df = new_pub_assist_df[(new_pub_assist_df['geographic_summary_level'] == 40)]
state_pub_assist_df.head()

,geographic_summary_level,state_FIPS_code,county_FIPS_code,place_FIPS_code,minor_civil_div_FIPS_code,consolidated_city_FIPS_code,primitive_geography_flag,functional_status_code,city,state,census_pop_2010,est_base_2010,est_pop_2010,est_pop_2011,est_pop_2012,est_pop_2013,est_pop_2014,est_pop_2015,est_pop_2016,geographic_level
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300,State
1105,40,2,0,0,0,0,0,A,Alaska,Alaska,710231,710249,714031,722713,731089,736879,736705,737709,741894,State
1452,40,4,0,0,0,0,0,A,Arizona,Arizona,6392017,6392301,6408312,6467163,6549634,6624617,6719993,6817565,6931071,State
1673,40,5,0,0,0,0,0,A,Arkansas,Arkansas,2915918,2916025,2921995,2939493,2950685,2958663,2966912,2977853,2988248,State
2848,40,6,0,0,0,0,0,A,California,California,37253956,37254522,37332685,37676861,38011074,38335203,38680810,38993940,39250017,State


In [24]:
# drop unneeded columns
state_pub_assist_df2 = state_pub_assist_df.drop(["census_pop_2010", "est_base_2010", "geographic_summary_level","state_FIPS_code", "county_FIPS_code", "place_FIPS_code", "minor_civil_div_FIPS_code", 
                                                 "consolidated_city_FIPS_code", "primitive_geography_flag", "functional_status_code", "city", "geographic_level"], axis=1)
state_pub_assist_df2

,state,est_pop_2010,est_pop_2011,est_pop_2012,est_pop_2013,est_pop_2014,est_pop_2015,est_pop_2016
0,Alabama,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1105,Alaska,714031,722713,731089,736879,736705,737709,741894
1452,Arizona,6408312,6467163,6549634,6624617,6719993,6817565,6931071
1673,Arkansas,2921995,2939493,2950685,2958663,2966912,2977853,2988248
2848,California,37332685,37676861,38011074,38335203,38680810,38993940,39250017
3925,Colorado,5048644,5118360,5189867,5267603,5349648,5448819,5540545
4616,Connecticut,3579899,3589893,3593795,3596003,3591873,3584730,3576452
4901,Delaware,899816,907924,916993,925395,934948,944076,952065
5025,District of Columbia,605183,620477,635327,649165,659005,670377,681170
5029,Florida,18849098,19096952,19344156,19582022,19888741,20244914,20612439


In [45]:
# rename year columns
state_pub_assist_df2.columns = ['state', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
state_pub_assist_df2

,state,2010,2011,2012,2013,2014,2015,2016
0,Alabama,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1105,Alaska,714031,722713,731089,736879,736705,737709,741894
1452,Arizona,6408312,6467163,6549634,6624617,6719993,6817565,6931071
1673,Arkansas,2921995,2939493,2950685,2958663,2966912,2977853,2988248
2848,California,37332685,37676861,38011074,38335203,38680810,38993940,39250017
3925,Colorado,5048644,5118360,5189867,5267603,5349648,5448819,5540545
4616,Connecticut,3579899,3589893,3593795,3596003,3591873,3584730,3576452
4901,Delaware,899816,907924,916993,925395,934948,944076,952065
5025,District of Columbia,605183,620477,635327,649165,659005,670377,681170
5029,Florida,18849098,19096952,19344156,19582022,19888741,20244914,20612439


In [46]:
# melt data from wide to long dataset
something_new_df = pd.melt(state_pub_assist_df2, id_vars=['state'], value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016'])
something_new_df

,state,variable,value
0,Alabama,2010,4785492
1,Alaska,2010,714031
2,Arizona,2010,6408312
3,Arkansas,2010,2921995
4,California,2010,37332685
5,Colorado,2010,5048644
6,Connecticut,2010,3579899
7,Delaware,2010,899816
8,District of Columbia,2010,605183
9,Florida,2010,18849098


In [47]:
# rename columns
something_new_df.columns = ['state', 'year', 'est_population']
something_new_df

,state,year,est_population
0,Alabama,2010,4785492
1,Alaska,2010,714031
2,Arizona,2010,6408312
3,Arkansas,2010,2921995
4,California,2010,37332685
5,Colorado,2010,5048644
6,Connecticut,2010,3579899
7,Delaware,2010,899816
8,District of Columbia,2010,605183
9,Florida,2010,18849098


In [48]:
# Add a column for WIC_usage, even if we don't have the data
something_new_df["WIC_usage"] = 'NaN'
something_new_df

,state,year,est_population,WIC_usage
0,Alabama,2010,4785492,NaN
1,Alaska,2010,714031,NaN
2,Arizona,2010,6408312,NaN
3,Arkansas,2010,2921995,NaN
4,California,2010,37332685,NaN
5,Colorado,2010,5048644,NaN
6,Connecticut,2010,3579899,NaN
7,Delaware,2010,899816,NaN
8,District of Columbia,2010,605183,NaN
9,Florida,2010,18849098,NaN


In [49]:
# Make the index the id column
something_new_df = something_new_df.reset_index(drop=False)
something_new_df.rename(columns = {"index":"id"}, inplace = True)
something_new_df

,id,state,year,est_population,WIC_usage
0,0,Alabama,2010,4785492,NaN
1,1,Alaska,2010,714031,NaN
2,2,Arizona,2010,6408312,NaN
3,3,Arkansas,2010,2921995,NaN
4,4,California,2010,37332685,NaN
5,5,Colorado,2010,5048644,NaN
6,6,Connecticut,2010,3579899,NaN
7,7,Delaware,2010,899816,NaN
8,8,District of Columbia,2010,605183,NaN
9,9,Florida,2010,18849098,NaN


In [31]:
# Create the Data Connection
rds_connection_string = "root:<password>@127.0.0.1/demographics_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [32]:
# Confirm tables
engine.table_names()

['min_wage', 'pub_assist']

In [51]:
# Load the Data to MySQL
something_new_df.to_sql(name='pub_assist', con=engine, if_exists='append', index=False)
min_wage_df2.to_sql(name='min_wage', con=engine, if_exists='append', index=False)